<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>



### Assignment 2
Welcome to Assignment 2. This will be fun. It is the first time you actually access external data from ApacheSpark.

#### You can also submit partial solutions

Just make sure you hit the play button on each cell from top to down. There are three functions you have to implement. Please also make sure than on each change on a function you hit the play button again on the corresponding cell to make it available to the rest of this notebook.

> Note: Replace **##** placeholder with the relevent parameters


In [44]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q


In [45]:
!pip install rklib 

In [46]:
import findspark
findspark.init()

In [47]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd()

In [48]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

This is the first function you have to implement. You are passed a dataframe object. We've also registered the dataframe in the ApacheSparkSQL catalog - so you can also issue queries against the "washing" table using "spark.sql()". Hint: To get an idea about the contents of the catalog you can use: spark.catalog.listTables().
So now it's time to implement your first function. You are free to use the dataframe API, SQL or RDD API. In case you want to use the RDD API just obtain the encapsulated RDD using "df.rdd". You can test the function by running one of the three last cells of this notebook, but please make sure you run the cells from top to down since some are dependant of each other...


In [49]:
#Please implement a function returning the number of rows in the dataframe
def count(spark, dataframe_name):
    #TODO Please enter your code here, you are not required to use the template code below
    #some reference: https://spark.apache.org/docs/latest/api/python/getting_started/quickstart_df.html#Working-with-SQL
    #some more help: https://www.w3schools.com/sql/sql_count_avg_sum.asp    
    
    # Register the DataFrame as a temporary table
    spark.sql(f"CREATE OR REPLACE TEMPORARY VIEW {dataframe_name} AS SELECT * FROM {dataframe_name}")

    # Use Spark SQL to count the number of rows
    result = spark.sql(f"SELECT COUNT(*) as cnt FROM {dataframe_name}")

    # Return the count as an integer
    return result.first().cnt


Now it's time to implement the second function. Please return an integer containing the number of fields (columns). The most easy way to get this is using the dataframe API. Hint: You might find the dataframe API documentation useful: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html


In [50]:
# def getNumberOfFields(## , ##):
#     #TODO Please enter your code here, you are not required to use the template code below
#     #some reference: https://spark.apache.org/docs/latest/api/python/getting_started/quickstart_df.html#Working-with-SQL
#     return len(df.##)
               
def getNumberOfFields(spark, dataframe_name):
    # Register the DataFrame as a temporary table
    spark.sql(f"CREATE OR REPLACE TEMPORARY VIEW {dataframe_name} AS SELECT * FROM {dataframe_name}")

    # Use Spark SQL to get the schema of the DataFrame
    result = spark.sql(f"DESCRIBE {dataframe_name}")

    # Count the number of fields (columns)
    num_fields = result.count()

    return num_fields

Finally, please implement a function which returns a (python) list of string values of the field names in this data frame. Hint: Just copy&past doesn't work because the auto-grader will create a random data frame for testing, so please use the data frame API as well. Again, this is the link to the documentation: https://spark.apache.org/docs/latest/api/python/getting_started/quickstart_df.html#Working-with-SQL


In [51]:
# def getFieldNames(## , ##):
#     #TODO Please enter your code here, you are not required to use the template code below
#     #some reference: https://spark.apache.org/docs/latest/api/python/getting_started/quickstart_df.html#Working-with-SQL
#     return df.##
    
def getFieldNames(spark, dataframe_name):
    # Register the DataFrame as a temporary table
    spark.sql(f"CREATE OR REPLACE TEMPORARY VIEW {dataframe_name} AS SELECT * FROM {dataframe_name}")

    # Use Spark SQL to get the schema of the DataFrame
    result = spark.sql(f"DESCRIBE {dataframe_name}")

    # Extract the field names from the result DataFrame
    field_names = [row.col_name for row in result.collect()]

    return field_names


Now it is time to grab a PARQUET file and create a dataframe out of it. Using SparkSQL you can handle it like a database.


In [52]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-SN0101EN-SkillsNetwork/labs/Week2_FSDS/washing.parquet
!mv washing.parquet?raw=true washing.parquet

--2023-11-17 23:33:30--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-SN0101EN-SkillsNetwork/labs/Week2_FSDS/washing.parquet
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 112048 (109K) [binary/octet-stream]
Saving to: ‘washing.parquet.2’

washing.parquet.2   100%[===================>] 109.42K  --.-KB/s    in 0.003s  

2023-11-17 23:33:30 (42.4 MB/s) - ‘washing.parquet.2’ saved [112048/112048]

mv: cannot stat 'washing.parquet?raw=true': No such file or directory


In [53]:
df = spark.read.parquet('washing.parquet')
df.createOrReplaceTempView('washing')
df.show()

+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|                 _id|                _rev|count|flowrate|fluidlevel|frequency|hardness|speed|temperature|           ts|voltage|
+--------------------+--------------------+-----+--------+----------+---------+--------+-----+-----------+-------------+-------+
|0d86485d0f88d1f9d...|1-57940679fb8a713...|    4|      11|acceptable|     null|      77| null|        100|1547808723923|   null|
|0d86485d0f88d1f9d...|1-15ff3a0b304d789...|    2|    null|      null|     null|    null| 1046|       null|1547808729917|   null|
|0d86485d0f88d1f9d...|1-97c2742b68c7b07...|    4|    null|      null|       71|    null| null|       null|1547808731918|    236|
|0d86485d0f88d1f9d...|1-eefb903dbe45746...|   19|      11|acceptable|     null|      75| null|         86|1547808738999|   null|
|0d86485d0f88d1f9d...|1-5f68b4c72813c25...|    7|    null|      null|       75|    null| null|   

The following cell can be used to test your count function


In [54]:
cnt = None
nof = None
fn = None

cnt = count(## , ##)
print(cnt)

SyntaxError: unexpected EOF while parsing (1905723318.py, line 6)

The following cell can be used to test your getNumberOfFields function


In [ ]:
nof = getNumberOfFields(## , ##)
print(nof)

The following cell can be used to test your getFieldNames function


In [ ]:
fn = getFieldNames(## , ##)
print(fn)

Congratulations, you are done. So please submit your solutions to the grader now.

# Start of Assignment-Submission

The first thing we need to do is to install a little helper library for submitting the solutions to the coursera grader:


In [ ]:
!rm -f rklib.py
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-SN0101EN-SkillsNetwork/rklib.py

Now it’s time to submit first solution. Please make sure that the token variable contains a valid submission token. You can obtain it from the coursera web page of the course using the grader section of this assignment.

Please specify you email address you are using with cousera as well.

<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-SN0101EN-SkillsNetwork/images/ML%20FS%20Grader%20screenshot1.png" width="500" alt="Skills Network Logo">
    </a>
</p>


In [55]:
from rklib import submit, submitAll
import json
    
key = "ho91rZfhdIXW4IU8"

if type(23) != type(cnt):
    raise ValueError('Please make sure that "cnt" is a number')

if type(23) != type(nof):
    raise ValueError('Please make sure that "nof" is a number')

if type([]) != type(fn):
    raise ValueError('Please make sure that "fn" is a list')

email = "mohamedskander.gasmi@esprit.tn"
token = "VdbV3Fd8ChPV5nDE" 
parts_data = {}
parts_data["2FjQw"] = json.dumps(cnt)
parts_data["j8gMs"] = json.dumps(nof)
parts_data["xaauC"] = json.dumps(fn)


submitAll(email, token, key, parts_data)

ImportError: cannot import name 'submit' from 'rklib' (/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/rklib/__init__.py)

<h3> Congratulations! You've successfully finished this lab. <h3/>

## Author(s)

<a href="https://www.linkedin.com/in/romeo-kienzler-089b4557/" target="_blank">Romeo Kienzler
</a>

## Other Contributor(s)
- <a href="https://www.linkedin.com/in/sowmyaa-gurusamy-b4a743202/" target="_blank">Sowmyaa Gurusamy</a>

- [Malika Singla](
https://www.linkedin.com/in/malika-goyal-04798622)
- [Lakshmi Holla](https://www.linkedin.com/in/lakshmi-holla-b39062149)

## Changelog

| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2023-09-15 | 1.0 | Sowmyaa Gurusamy | Created initial version |
|

<center><h3> © IBM Corporation 2023. All rights reserved. <h3/></center>
